In [3]:
import Utils.DiaSourceTools as DSTools
import lsst.afw.display.ds9 as ds9
import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom


In [4]:
DATADIR="/datadec/cppm/jpreyes/CFHT_Test/"
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_daytest/"
directory = DATADIR+"/detect_testSN_par_2/"
butler = dafPersist.Butler(directory) 

In [23]:

def getStamp(ra, dec,  exposure, size=20, offset=0):
    
    
    sourceRa = afwGeom.Angle(ra,  afwGeom.degrees)
    sourceDec = afwGeom.Angle(dec,  afwGeom.degrees)

    wcs = exposure.getWcs()

    Center = afwGeom.Point2I(wcs.skyToPixel(sourceRa, sourceDec))

    height= size/2
    width= size/2
    
    print Center


    centerX= Center[0]
    centerY= Center[1]

    Begin = afwGeom.Point2D(centerX - height, centerY - width)
    Begin = afwGeom.Point2I(Begin)

    End = afwGeom.Point2D(centerX + height+1, centerY + width+1)
    End = afwGeom.Point2I(End)



    ExpOrig = afwGeom.Point2I(exposure.getX0(), exposure.getY0())



    correctedBegin = Begin- ExpOrig
    correctedEnd = End - ExpOrig

    correctedBegin= afwGeom.Point2I(correctedBegin.getX()-offset,correctedBegin.getY()-offset )
    correctedEnd = afwGeom.Point2I(correctedEnd.getX()+offset,correctedEnd.getY()+offset )


    bboxT = afwGeom.Box2I(correctedBegin,correctedEnd) 



    #bboxT = bbox
        #print bboxT.toString, centerX, centerY
    return exposure.Factory(exposure,bboxT, True)


In [29]:
patch = '14,6'
ra = 215.18155833333333
dec = 52.191063888888884
date = '2006-02-03'

dataId= {"filter":'r',"tract":0, "patch":patch, 'date':date}
exposure = butler.get("dayDiff_differenceExp", dataId=dataId)


stamp = getStamp(ra, dec, exposure, size=20)

ds9.mtv(stamp, frame = 1)
ds9.mtv(exposure, frame=2)
ds9.dot('o', 14840, 6059, frame=2)

(14840, 6059)


In [63]:
def is_near_border(source, exposure):
    wcs = exposure.getWcs()
    
    point = afwGeom.Point2I(wcs.skyToPixel(source.get("coord_ra"), source.get("coord_dec")))
    #print point
    
    bbox = exposure.getBBox()
    
    b_max_x =  bbox.getMaxX() - 50
    b_min_x =  bbox.getMinX() + 50
    b_max_y =  bbox.getMaxY() - 50
    b_min_y =  bbox.getMinY() + 50
    
    if point.getX() > b_max_x or point.getX() < b_min_x or point.getY() > b_max_y or point.getY() < b_min_y:
        print point, b_max_x, b_min_x, b_max_y, b_min_y
        print True
    else:
        print False
    
    #print bbox

In [65]:
dataId= {"filter":'r',"tract":0, "patch":'10,10', 'date':date}

sources = butler.get("dayDiff_class_diaSrc", dataId=dataId)
exposure = butler.get("dayDiff_differenceExp", dataId=dataId)


In [66]:
for src in sources:
    is_near_border(src, exposure)

(10304, 9919) 11049 9950 11049 9950
True
(10339, 9919) 11049 9950 11049 9950
True
(10897, 9923) 11049 9950 11049 9950
True
(10049, 9923) 11049 9950 11049 9950
True
(10400, 9923) 11049 9950 11049 9950
True
(10737, 9929) 11049 9950 11049 9950
True
(9943, 9933) 11049 9950 11049 9950
True
(10357, 9945) 11049 9950 11049 9950
True
(10503, 9947) 11049 9950 11049 9950
True
False
False
False
False
False
(9919, 10006) 11049 9950 11049 9950
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
(9935, 10503) 11049 9950 11049 9950
True
False
False
False
False
False
False
(9933, 10550) 11049 9950 11049 9950
True
False
False
False
False
False
False
False
False
(9919, 10669) 11049 9950 11049 9950
True
False
False
False
False
False
False
False
False
False
False
False


In [40]:
bbox = exposure.getBBox()
print bbox.getMaxX(), bbox.getMinX()
print bbox.getMaxY(), bbox.getMinY()

15099 13900
7099 5900


In [44]:
wcs = exposure.getWcs()
    
point = afwGeom.Point2I(wcs.skyToPixel(sources[0].get("coord_ra"), sources[0].get("coord_dec")))